验证环境

In [ ]:
from ultralytics import YOLO
# 加载预训练的 YOLOv11n 模型
model_path =  '/home/szh/work/Weed_Detection/ultralytics-main/runs/detect/train8/weights/best.pt'#'/home/szh/work/Weed_Detection/ultralytics-main/yolo11n.pt'
model = YOLO(model_path)
source = '/home/szh/work/Weed_Detection/test/images/19117.png' #'/home/szh/work/Weed_Detection/cat.jpg' #更改为自己的图片路径
# 运行推理，并附加参数
model.predict(source, save=True)

数据集格式转yolo格式

In [6]:
import os
import json
import math

# 定义目标label的类ID
LABELS = {
    "weed": 0,  # 其他所有的 label 统一为 "weed"，class_id 为 0
    "mq": 1     # mq 对应的 class_id 为 1
}

def convert_to_yolo_format(points, img_width, img_height):
    # YOLO格式: class_id center_x center_y width height
    # points 是一个包含两个坐标 [(x1, y1), (x2, y2)]，表示的是矩形的两个点
    x_1, y_1 = points[0]
    x_2, y_2 = points[1]

    r = math.sqrt((x_2 - x_1) ** 2 + (y_2 - y_1) ** 2)
    # 外接正方形的左上角和右下角
    x1 = x_1 - r
    y1 = y_1 - r
    
    x2 = x_1 + r
    y2 = y_1 + r


    # 将矩形的两个点转换为边界框的中心点和宽高
    center_x = (x1 + x2) / 2.0 / img_width
    center_y = (y1 + y2) / 2.0 / img_height

    width = (2*r) / img_width
    height = (2*r) / img_height

    return center_x, center_y, width, height

def process_json_label(json_file, output_dir):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    img_width = data['imageWidth']
    img_height = data['imageHeight']
    
    # 获取图像的文件名 (无后缀)
    image_filename = os.path.splitext(os.path.basename(data['imagePath']))[0]

    # 输出的YOLO格式标签文件路径
    yolo_txt_path = os.path.join(output_dir, f"{image_filename}.txt")
    
    # 打开文件写入YOLO格式的标签
    with open(yolo_txt_path, 'w') as yolo_file:
        for shape in data['shapes']:
            label = shape['label']
            
            # 只检测 "mq"，其余的都统一成 "weed"
            if label == 'mq':
                class_id = LABELS['mq']
            else:
                class_id = LABELS['weed']
            
            # 解析 points，转换成 YOLO 格式
            points = shape['points']
            center_x, center_y, width, height = convert_to_yolo_format(points, img_width, img_height)
            
            # 写入文件，格式: class_id center_x center_y width height
            yolo_file.write(f"{class_id} {center_x} {center_y} {width} {height}\n")

def convert_dataset(json_dir, output_dir):
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 遍历所有的json文件
    for filename in os.listdir(json_dir):
        if filename.endswith('.json'):
            json_file = os.path.join(json_dir, filename)
            process_json_label(json_file, output_dir)

if __name__ == "__main__":
    # 输入的JSON标注文件目录
    json_dir = "/home/szh/work/Weed_Detection/train/labels"
    # 输出的YOLO格式标签的目录
    output_dir = "/home/szh/work/Weed_Detection/train/labels_process"
    
    convert_dataset(json_dir, output_dir)

数据预处理

In [ ]:
import os
import shutil

def process_files(input_path, output_path):
    total_deleted_lines = 0
    log_entries = []
    
    # 确保输出目录存在
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # 遍历输入路径下的所有 txt 文件
    for filename in os.listdir(input_path):
        if filename.endswith(".txt"):
            input_file_path = os.path.join(input_path, filename)
            output_file_path = os.path.join(output_path, filename)
            
            with open(input_file_path, "r") as file:
                lines = file.readlines()

            # 保存更新后的内容
            updated_lines = []
            deleted_lines_count = 0
            
            for i, line in enumerate(lines):
                parts = line.strip().split()
                if len(parts) >= 4:
                    fourth_value = float(parts[3])  # 取第四个数字
                    x_value = float(parts[1])
                    y_value = float(parts[2])
                    if fourth_value < 0.025:
                        deleted_lines_count += 1
                        log_entries.append(f"File: {filename}, Line {i+1}: Deleted (Fourth value < 0.025)")
                    else:
                        updated_lines.append(line)
            
            # 保存更新后的文件到输出路径
            with open(output_file_path, "w") as file:
                file.writelines(updated_lines)

            total_deleted_lines += deleted_lines_count

    # 输出日志和删除的行数
    if log_entries:
        for log in log_entries:
            print(log)
    print(f"Total lines deleted across all files: {total_deleted_lines}")

# Example usage:
input_path = "/home/szh/work/Weed_Detection/train/data_Division_preprocess/train_Augment/labels"  # 输入目录路径
output_path = "/home/szh/work/Weed_Detection/train/data_Division_preprocess/train_Augment/labels_pre"  # 输出目录路径
process_files(input_path, output_path)


数据增强

In [ ]:
# -*- coding: utf-8 -*- 
""" 
Created on 2023-04-01 9:08 
@author: Fan yi ming 
Func: 对于目标检测的数据增强[YOLO]（特点是数据增强后标签也要更改）
review：常用的数据增强方式； 
        1.翻转：左右和上下翻转，随机翻转 
        2.随机裁剪，图像缩放 
        3.改变色调 
        4.添加噪声 
注意： boxes的标签和坐标一个是int，一个是float，存放的时候要注意处理方式。 
参考：https://github.com/REN-HT/Data-Augmentation/blob/main/data_augmentation.py 
"""
import torch
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import os
import random
random.seed(0)
 
 
class DataAugmentationOnDetection:
    def __init__(self):
        super(DataAugmentationOnDetection, self).__init__()
 
    # 以下的几个参数类型中，image的类型全部如下类型
    # 参数类型： image：Image.open(path)
    def resize_keep_ratio(self, image, boxes, target_size):
        """
            参数类型： image：Image.open(path)， boxes:Tensor， target_size:int
            功能：将图像缩放到size尺寸，调整相应的boxes,同时保持长宽比（最长的边是target size
        """
        old_size = image.size[0:2]  # 原始图像大小
        # 取最小的缩放比例
        ratio = min(float(target_size) / (old_size[i]) for i in range(len(old_size)))  # 计算原始图像宽高与目标图像大小的比例，并取其中的较小值
        new_size = tuple([int(i * ratio) for i in old_size])  # 根据上边求得的比例计算在保持比例前提下得到的图像大小
        # boxes 不用变化，因为是等比例变化
        return image.resize(new_size, Image.BILINEAR), boxes
 
    def resizeDown_keep_ratio(self, image, boxes, target_size):
        """ 与上面的函数功能类似，但它只降低图片的尺寸，不会扩大图片尺寸"""
        old_size = image.size[0:2]  # 原始图像大小
        # 取最小的缩放比例
        ratio = min(float(target_size) / (old_size[i]) for i in range(len(old_size)))  # 计算原始图像宽高与目标图像大小的比例，并取其中的较小值
        ratio = min(ratio, 1)
        new_size = tuple([int(i * ratio) for i in old_size])  # 根据上边求得的比例计算在保持比例前提下得到的图像大小
 
        # boxes 不用变化，因为是等比例变化
        return image.resize(new_size, Image.BILINEAR), boxes
 
    def resize(self, img, boxes, size):
        # ---------------------------------------------------------
        # 类型为 img=Image.open(path)，boxes:Tensor，size:int
        # 功能为：将图像长和宽缩放到指定值size，并且相应调整boxes
        # ---------------------------------------------------------
        return img.resize((size, size), Image.BILINEAR), boxes
 
    def random_flip_horizon(self, img, boxes, h_rate=1):
        # -------------------------------------
        # 随机水平翻转
        # -------------------------------------
        if np.random.random() < h_rate:
            transform = transforms.RandomHorizontalFlip(p=1)
            img = transform(img)
            if len(boxes) > 0:
                x = 1 - boxes[:, 1]
                boxes[:, 1] = x
        return img, boxes
 
    def random_flip_vertical(self, img, boxes, v_rate=1):
        # 随机垂直翻转
        if np.random.random() < v_rate:
            transform = transforms.RandomVerticalFlip(p=1)
            img = transform(img)
            if len(boxes) > 0:
                y = 1 - boxes[:, 2]
                boxes[:, 2] = y
        return img, boxes
 
    def center_crop(self, img, boxes, target_size=None):
        # -------------------------------------
        # 中心裁剪 ，裁剪成 (size, size) 的正方形, 仅限图形，w,h
        # 这里用比例是很难算的，转成x1,y1, x2, y2格式来计算
        # -------------------------------------
        w, h = img.size
        size = min(w, h)
        if len(boxes) > 0:
            # 转换到xyxy格式
            label = boxes[:, 0].reshape([-1, 1])
            x_, y_, w_, h_ = boxes[:, 1], boxes[:, 2], boxes[:, 3], boxes[:, 4]
            x1 = (w * x_ - 0.5 * w * w_).reshape([-1, 1])
            y1 = (h * y_ - 0.5 * h * h_).reshape([-1, 1])
            x2 = (w * x_ + 0.5 * w * w_).reshape([-1, 1])
            y2 = (h * y_ + 0.5 * h * h_).reshape([-1, 1])
            boxes_xyxy = torch.cat([x1, y1, x2, y2], dim=1)
            # 边框转换
            if w > h:
                boxes_xyxy[:, [0, 2]] = boxes_xyxy[:, [0, 2]] - (w - h) / 2
            else:
                boxes_xyxy[:, [1, 3]] = boxes_xyxy[:, [1, 3]] - (h - w) / 2
            in_boundary = [i for i in range(boxes_xyxy.shape[0])]
            for i in range(boxes_xyxy.shape[0]):
                # 判断x是否超出界限
                if (boxes_xyxy[i, 0] < 0 and boxes_xyxy[i, 2] < 0) or (boxes_xyxy[i, 0] > size and boxes_xyxy[i, 2] > size):
                    in_boundary.remove(i)
                # 判断y是否超出界限
                elif (boxes_xyxy[i, 1] < 0 and boxes_xyxy[i, 3] < 0) or (boxes_xyxy[i, 1] > size and boxes_xyxy[i, 3] > size):
                    in_boundary.append(i)
            boxes_xyxy = boxes_xyxy[in_boundary]
            boxes = boxes_xyxy.clamp(min=0, max=size).reshape([-1, 4])  # 压缩到固定范围
            label = label[in_boundary]
            # 转换到YOLO格式
            x1, y1, x2, y2 = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]
            xc = ((x1 + x2) / (2 * size)).reshape([-1, 1])
            yc = ((y1 + y2) / (2 * size)).reshape([-1, 1])
            wc = ((x2 - x1) / size).reshape([-1, 1])
            hc = ((y2 - y1) / size).reshape([-1, 1])
            boxes = torch.cat([xc, yc, wc, hc], dim=1)
        # 图像转换 
        transform = transforms.CenterCrop(size) 
        img = transform(img) 
        if target_size: 
            img = img.resize((target_size, target_size), Image.BILINEAR) 
        if len(boxes) > 0: 
            return img, torch.cat([label.reshape([-1, 1]), boxes], dim=1) 
        else: 
            return img, boxes 
  
    # ------------------------------------------------------
    # 以下img皆为Tensor类型
    # ------------------------------------------------------
 
    def random_bright(self, img, u=120, p=1): 
        # -------------------------------------
        # 随机亮度变换 
        # -------------------------------------
        if np.random.random() < p:
            alpha=np.random.uniform(-u, u)/255
            img += alpha
            img=img.clamp(min=0.0, max=1.0)
        return img
 
    def random_contrast(self, img, lower=0.5, upper=1.5, p=1):
        # -------------------------------------
        # 随机增强对比度 
        # -------------------------------------
        if np.random.random() < p:
            alpha=np.random.uniform(lower, upper)
            img*=alpha
            img=img.clamp(min=0, max=1.0)
        return img
 
    def random_saturation(self, img,lower=0.5, upper=1.5, p=1):
        # 随机饱和度变换，针对彩色三通道图像，中间通道乘以一个值
        if np.random.random() < p: 
            alpha=np.random.uniform(lower, upper) 
            img[1]=img[1]*alpha 
            img[1]=img[1].clamp(min=0,max=1.0) 
        return img 
 
    def add_gasuss_noise(self, img, mean=0, std=0.1): 
        noise=torch.normal(mean,std,img.shape) 
        img+=noise 
        img=img.clamp(min=0, max=1.0) 
        return img 
 
    def add_salt_noise(self, img):
        noise=torch.rand(img.shape)
        alpha=np.random.random()/5 + 0.7
        img[noise[:,:,:]>alpha]=1.0
        return img
 
    def add_pepper_noise(self, img):
        noise=torch.rand(img.shape)
        alpha=np.random.random()/5 + 0.7
        img[noise[:, :, :]>alpha]=0
        return img
 
 
def plot_pics(img, boxes):
    # 显示图像和候选框，img是Image.Open()类型, boxes是Tensor类型
    plt.imshow(img)
    label_colors = [(213, 110, 89)]
    w, h = img.size
    for i in range(boxes.shape[0]):
        box = boxes[i, 1:]
        xc, yc, wc, hc = box
        x = w * xc - 0.5 * w * wc
        y = h * yc - 0.5 * h * hc
        box_w, box_h = w * wc, h * hc
        plt.gca().add_patch(plt.Rectangle(xy=(x, y), width=box_w, height=box_h,
                                          edgecolor=[c / 255 for c in label_colors[0]],
                                          fill=False, linewidth=2))
    plt.show()
 
def get_image_list(image_path):
    # 根据图片文件，查找所有图片并返回列表
    files_list = []
    for root, sub_dirs, files in os.walk(image_path):
        for special_file in files:
            special_file = special_file[0: len(special_file)]
            files_list.append(special_file)
    return files_list
 
def get_label_file(label_path, image_name):
    # 根据图片信息，查找对应的label
    fname = os.path.join(label_path, image_name[0: len(image_name)-4]+".txt")
    data2 = []
    if not os.path.exists(fname):
        return data2
    if os.path.getsize(fname) == 0:
        return data2
    else:
        with open(fname, 'r', encoding='utf-8') as infile:
            # 读取并转换标签
            for line in infile:
                data_line = line.strip("\n").split()
                data2.append([float(i) for i in data_line])
    return data2
 
 
def save_Yolo(img, boxes, save_path, prefix, image_name):
    # img: 需要时Image类型的数据， prefix 前缀
    # 将结果保存到save path指示的路径中
    if not os.path.exists(save_path) or \
            not os.path.exists(os.path.join(save_path, "images")):
        os.makedirs(os.path.join(save_path, "images"))
        os.makedirs(os.path.join(save_path, "labels"))
    try:
        img.save(os.path.join(save_path, "images", prefix + image_name))
        with open(os.path.join(save_path, "labels", prefix + image_name[0:len(image_name)-4] + ".txt"), 'w', encoding="utf-8") as f:
            if len(boxes) > 0:  # 判断是否为空
                # 写入新的label到文件中
                for data in boxes:
                    str_in = ""
                    for i, a in enumerate(data):
                        if i == 0:
                            str_in += str(int(a))
                        else:
                            str_in += " " + str(float(a))
                    f.write(str_in + '\n')
    except:
        print("ERROR: ", image_name, " is bad.")
 
 
def runAugumentation(image_path, label_path, save_path):
    image_list = get_image_list(image_path)
    for image_name in image_list:
        print("dealing: " + image_name)
        img = Image.open(os.path.join(image_path, image_name))
        boxes = get_label_file(label_path, image_name)
        boxes = torch.tensor(boxes)
        # 下面是执行的数据增强功能，可自行选择
        # Image类型的参数
        DAD = DataAugmentationOnDetection()
 
        """ 尺寸变换   """
        # 缩小尺寸
        # t_img, t_boxes = DAD.resizeDown_keep_ratio(img, boxes, 1024)
        # save_Yolo(t_img, boxes, save_path, prefix="rs_", image_name=image_name)
        # 水平旋转
        t_img, t_boxes = DAD.random_flip_horizon(img, boxes.clone())
        save_Yolo(t_img, t_boxes, save_path, prefix="fh_", image_name=image_name)
        # 竖直旋转
        t_img, t_boxes = DAD.random_flip_vertical(img, boxes.clone())
        save_Yolo(t_img, t_boxes, save_path, prefix="fv_", image_name=image_name)
        # center_crop
        t_img, t_boxes = DAD.center_crop(img, boxes.clone(), 1024)
        save_Yolo(t_img, t_boxes, save_path, prefix="cc_", image_name=image_name)
 
        """ 图像变换，用tensor类型"""
        to_tensor = transforms.ToTensor()
        to_image = transforms.ToPILImage()
        img = to_tensor(img)
 
        # random_bright
        t_img, t_boxes = DAD.random_bright(img.clone()), boxes
        save_Yolo(to_image(t_img), boxes, save_path, prefix="rb_", image_name=image_name)
        # random_contrast 对比度变化
        t_img, t_boxes = DAD.random_contrast(img.clone()), boxes
        save_Yolo(to_image(t_img), boxes, save_path, prefix="rc_", image_name=image_name)
        # random_saturation 饱和度变化
        t_img, t_boxes = DAD.random_saturation(img.clone()), boxes
        save_Yolo(to_image(t_img), boxes, save_path, prefix="rs_", image_name=image_name)
        # 高斯噪声
        # t_img, t_boxes = DAD.add_gasuss_noise(img.clone()), boxes
        # save_Yolo(to_image(t_img), boxes, save_path, prefix="gn_", image_name=image_name)
        # add_salt_noise
        # t_img, t_boxes = DAD.add_salt_noise(img.clone()), boxes
        # save_Yolo(to_image(t_img), boxes, save_path, prefix="sn_", image_name=image_name)
        # add_pepper_noise
        # t_img, t_boxes = DAD.add_pepper_noise(img.clone()), boxes
        # save_Yolo(to_image(t_img), boxes, save_path, prefix="pn_", image_name=image_name)
 
        print("end:     " + image_name)
 
 
if __name__ == '__main__':
    # 图像和标签文件夹
    image_path = '/home/szh/work/Weed_Detection/train/data_Division_preprocess/train/images'
    label_path = '/home/szh/work/Weed_Detection/train/data_Division_preprocess/train/labels'
    save_path = '/home/szh/work/Weed_Detection/train/data_Division_preprocess/train_Augment'         # 结果保存位置路径，可以是一个不存在的文件夹
    # 运行
    runAugumentation(image_path, label_path, save_path)

数据集的划分  训练集：验证集：测试集=

In [ ]:
import os
import shutil
import random
 
random.seed(0)  #随机种子，可自选开启
def split_data(file_path, label_path, new_file_path, train_rate, val_rate, test_rate):
	images = os.listdir(file_path)
	labels = os.listdir(label_path)
	images_no_ext = {os.path.splitext(image)[0]: image for image in images}
	labels_no_ext = {os.path.splitext(label)[0]: label for label in labels}
	matched_data = [(img, images_no_ext[img], labels_no_ext[img]) for img in images_no_ext if img in labels_no_ext]
 
	unmatched_images = [img for img in images_no_ext if img not in labels_no_ext]
	unmatched_labels = [label for label in labels_no_ext if label not in images_no_ext]
	if unmatched_images:
		print("未匹配的图片文件:")
		for img in unmatched_images:
			print(images_no_ext[img])
	if unmatched_labels:
		print("未匹配的标签文件:")
		for label in unmatched_labels:
			print(labels_no_ext[label])
 
	random.shuffle(matched_data)
	total = len(matched_data)
	train_data = matched_data[:int(train_rate * total)]
	val_data = matched_data[int(train_rate * total):int((train_rate + val_rate) * total)]
	test_data = matched_data[int((train_rate + val_rate) * total):]
 
	# 处理训练集
	for img_name, img_file, label_file in train_data:
		old_img_path = os.path.join(file_path, img_file)
		old_label_path = os.path.join(label_path, label_file)
		new_img_dir = os.path.join(new_file_path, 'train', 'images')
		new_label_dir = os.path.join(new_file_path, 'train', 'labels')
		os.makedirs(new_img_dir, exist_ok=True)
		os.makedirs(new_label_dir, exist_ok=True)
		shutil.copy(old_img_path, os.path.join(new_img_dir, img_file))
		shutil.copy(old_label_path, os.path.join(new_label_dir, label_file))
	# 处理验证集
	for img_name, img_file, label_file in val_data:
		old_img_path = os.path.join(file_path, img_file)
		old_label_path = os.path.join(label_path, label_file)
		new_img_dir = os.path.join(new_file_path, 'val', 'images')
		new_label_dir = os.path.join(new_file_path, 'val', 'labels')
		os.makedirs(new_img_dir, exist_ok=True)
		os.makedirs(new_label_dir, exist_ok=True)
		shutil.copy(old_img_path, os.path.join(new_img_dir, img_file))
		shutil.copy(old_label_path, os.path.join(new_label_dir, label_file))
	# 处理测试集
	for img_name, img_file, label_file in test_data:
		old_img_path = os.path.join(file_path, img_file)
		old_label_path = os.path.join(label_path, label_file)
		new_img_dir = os.path.join(new_file_path, 'test', 'images')
		new_label_dir = os.path.join(new_file_path, 'test', 'labels')
		os.makedirs(new_img_dir, exist_ok=True)
		os.makedirs(new_label_dir, exist_ok=True)
		shutil.copy(old_img_path, os.path.join(new_img_dir, img_file))
		shutil.copy(old_label_path, os.path.join(new_label_dir, label_file))
	print("数据集已划分完成")
 
if __name__ == '__main__':
	file_path = "/home/szh/work/Weed_Detection/train/images"  # 图片文件夹
	label_path = '/home/szh/work/Weed_Detection/train/labels'  # 标签文件夹
	new_file_path = '/home/szh/work/Weed_Detection/train/data_Division' # 新数据存放位置
	split_data(file_path, label_path, new_file_path, train_rate=0.8, val_rate=0.2, test_rate=0)

训练模型

In [ ]:
from ultralytics import YOLO
if __name__ == '__main__': 
    model = YOLO('ultralytics/cfg/models/11/yolo11n.yaml')  # 从YAML建立一个新模型
    model_path = '/home/szh/work/Weed_Detection/ultralytics-main/yolo11n.pt'
    model.load(model_path)
    print("开始训练： ")
    # 训练模型
    results = model.train(data='data.yaml',
                      epochs=600, imgsz=640, device=0, optimizer='SGD', workers=8, batch=64, amp=False)

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt

if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/11/yolo11n.yaml')  # 从YAML建立一个新模型
    model_path = '/home/szh/work/Weed_Detection/ultralytics-main/yolo11n.pt'
    model.load(model_path)
    print("开始训练： ")

    # 训练模型并存储结果
    results = model.train(data='data.yaml',
                          epochs=300,
                          imgsz=640,
                          device=0,
                          optimizer='SGD',
                          workers=8,
                          batch=64,
                          amp=False)

    # 获取训练结果
    metrics = results.results

    # 从metrics中提取数据
    epochs = range(1, len(metrics['metrics/precision']) + 1)
    train_loss = metrics['train/loss']
    val_loss = metrics['val/loss']
    precision = metrics['metrics/precision']
    recall = metrics['metrics/recall']
    mAP_05 = metrics['metrics/mAP_0.5']
    mAP_05_095 = metrics['metrics/mAP_0.5:0.95']

    # 绘制loss曲线
    plt.figure(figsize=(12, 6))
    plt.subplot(2, 2, 1)
    plt.plot(epochs, train_loss, label='Train Loss')
    plt.plot(epochs, val_loss, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    # 绘制准确率曲线
    plt.subplot(2, 2, 2)
    plt.plot(epochs, precision, label='Precision')
    plt.plot(epochs, recall, label='Recall')
    plt.xlabel('Epoch')
    plt.ylabel('Metrics')
    plt.legend()
    plt.title('Precision and Recall')

    # 绘制mAP曲线
    plt.subplot(2, 2, 3)
    plt.plot(epochs, mAP_05, label='mAP@0.5')
    plt.plot(epochs, mAP_05_095, label='mAP@0.5:0.95')
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.legend()
    plt.title('Mean Average Precision (mAP)')

    plt.tight_layout()
    plt.show()

    # 打印每个epoch后的loss和准确率信息以及所用时间
    for i, (t_loss, v_loss, prec, rec, map50, map5095) in enumerate(zip(train_loss, val_loss, precision, recall, mAP_05, mAP_05_095)):
        print(f"Epoch {i+1}: Train Loss: {t_loss:.4f}, Val Loss: {v_loss:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, mAP@0.5: {map50:.4f}, mAP@0.5:0.95: {map5095:.4f}")

    # 打印训练总时间
    print(f"\nTotal training time: {results.t['total'] / 60:.2f} minutes")

模型训练结果绘图

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 读取 CSV 文件的数据
# 这里假设 result.csv 文件存放在你的工作目录下，或者提供文件的绝对路径
file_path = '/home/szh/work/Weed_Detection/ultralytics-main/runs/detect/train79/results.csv'

# 使用 pandas 读取 CSV 文件
df = pd.read_csv(file_path)

# 绘制训练和验证损失
plt.figure(figsize=(12, 8))

# 创建一个图，绘制训练和验证损
plt.subplot(2, 2, 1)
plt.plot(df['epoch'], df['train/box_loss'], label='train/box_loss', color='blue')
plt.plot(df['epoch'], df['val/box_loss'], label='val/box_loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('Box Loss')
plt.title('Box Loss')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(df['epoch'], df['train/cls_loss'], label='train/cls_loss', color='blue')
plt.plot(df['epoch'], df['val/cls_loss'], label='val/cls_loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('Class Loss')
plt.title('Class Loss')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(df['epoch'], df['train/dfl_loss'], label='train/dfl_loss', color='blue')
plt.plot(df['epoch'], df['val/dfl_loss'], label='val/dfl_loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('DFL Loss')
plt.title('DFL Loss')
plt.legend()

# 绘制其他指标（Precision, Recall, mAP50等）
plt.subplot(2, 2, 4)
plt.plot(df['epoch'], df['metrics/precision(B)'], label='Precision', color='green')
plt.plot(df['epoch'], df['metrics/recall(B)'], label='Recall', color='orange')
plt.plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP50', color='purple')
plt.plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP50-95', color='brown')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Precision, Recall, mAP')
plt.legend()

# 调整布局
plt.tight_layout()

# 显示图形
plt.show()


测试集测试

In [21]:
import pandas as pd
import os
from ultralytics import YOLO
import pdb
import torch

# os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# torch.cuda.set_device(3)

# 加载训练好的模型
best_model_path = '/home/szh/work/Weed_Detection/ultralytics-main/runs/detect/split_4_dataset/weights/best.pt'
model = YOLO(best_model_path)

# 测试图像目录
source_dir = '/home/szh/work/Weed_Detection/test/images'  # 修改为自己的图片路径

# 获取目录下所有图像文件
image_files = [f for f in os.listdir(source_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
print(len(image_files))

# 准备保存的列表
output_data = []

# 遍历每个图像文件并进行推理
j=0
for image_file in image_files:
    # 提取文件名作为 image_id（去掉扩展名）
    image_id = os.path.splitext(image_file)[0]
    print("image_id = ",image_id)

    # 生成图像的完整路径
    image_path = os.path.join(source_dir, image_file)
    
    # 运行推理
    results = model.predict(image_path)
    
    # 遍历推理结果
    for result in results:
        boxes = result.boxes  # 获取预测到的所有框
        for i, box in enumerate(boxes):
            class_id = int(box.cls.item())  # 类别: 0 为 'weed', 1 为 'mq'
            
            # 获取 xyxy 坐标
            xyxy = box.xyxy[0].cpu().numpy()  # 将 Tensor 转换为 NumPy 数组
            
            # 提取坐标
            x_min = int(xyxy[0])  # 左上角 X 坐标
            y_min = int(xyxy[1])  # 左上角 Y 坐标
            x_max = int(xyxy[2])  # 右下角 X 坐标
            y_max = int(xyxy[3])  # 右下角 Y 坐标
            
            # 计算宽度和高度
            width = x_max - x_min
            height = y_max - y_min
            
            # 添加到输出数据
            j=j+1
            output_data.append([j, image_id, class_id, x_min, y_min, width, height])
            

# 将数据保存为 CSV 格式
df = pd.DataFrame(output_data, columns=['ID', 'image_id', 'class_id', 'x_min', 'y_min', 'width', 'height'])
df.to_csv('/home/szh/work/Weed_Detection/test/output_predictions_split_4.csv', index=False)

print("预测结果已保存到 '/home/szh/work/Weed_Detection/test/output_predictions_split_4.csv'")

486
image_id =  3272


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


5折交叉验证非极大值抑制集成测试

In [ ]:
import pandas as pd
import os
from ultralytics import YOLO
import torch
from torchvision.ops import nms  # NMS function from PyTorch

# Paths to your five models
model_paths = [
    '/home/szh/work/Weed_Detection/ultralytics-main/runs/detect/split_1_dataset/weights/best.pt',
    '/home/szh/work/Weed_Detection/ultralytics-main/runs/detect/split_2_dataset/weights/best.pt',
    '/home/szh/work/Weed_Detection/ultralytics-main/runs/detect/split_3_dataset/weights/best.pt',
    '/home/szh/work/Weed_Detection/ultralytics-main/runs/detect/split_4_dataset/weights/best.pt',
    '/home/szh/work/Weed_Detection/ultralytics-main/runs/detect/split_5_dataset/weights/best.pt'
]

# Load models
models = [YOLO(path) for path in model_paths]

# Directory for test images
source_dir = '/home/szh/work/Weed_Detection/test/images'
image_files = [f for f in os.listdir(source_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

output_data = []
j = 0

# Define NMS threshold
nms_threshold = 0.7

# Process each image
for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    image_path = os.path.join(source_dir, image_file)
    
    # Collect predictions from all models
    boxes_list = []
    scores_list = []
    classes_list = []

    for model in models:
        results = model.predict(image_path)
        
        for result in results:
            boxes = result.boxes
            for box in boxes:
                class_id = int(box.cls.item())
                xyxy = box.xyxy[0].cpu().numpy()
                score = float(box.conf)  # Confidence score

                x_min, y_min, x_max, y_max = map(int, xyxy)
                boxes_list.append([x_min, y_min, x_max, y_max])
                scores_list.append(score)
                classes_list.append(class_id)

    # Check if there are any boxes to process
    if len(boxes_list) > 0:
        # Convert lists to tensors for NMS processing
        boxes_tensor = torch.tensor(boxes_list, dtype=torch.float32)
        scores_tensor = torch.tensor(scores_list)
        classes_tensor = torch.tensor(classes_list)

        # Apply NMS
        keep_indices = nms(boxes_tensor, scores_tensor, nms_threshold)

        # Save NMS-filtered results
        for idx in keep_indices:
            j += 1
            x_min, y_min, x_max, y_max = map(int, boxes_tensor[idx])
            class_id = int(classes_tensor[idx].item())
            width = x_max - x_min
            height = y_max - y_min
            output_data.append([j, image_id, class_id, x_min, y_min, width, height])

# Save results to CSV
df = pd.DataFrame(output_data, columns=['ID', 'image_id', 'class_id', 'x_min', 'y_min', 'width', 'height'])
output_csv_path = '/home/szh/work/Weed_Detection/test/output_predictions_ensemble_0.7.csv'
df.to_csv(output_csv_path, index=False)

print(f"Ensemble prediction results saved to '{output_csv_path}'")
